In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

# The N-Queens-Problem as a CSP

The function `create_csp(n)` takes a natural number `n` as argument and returns
a *constraint satisfaction problem* that encodes the 
[`n`-queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle).
    
A constraint satisfaction problem $\mathcal{P}$ is a triple of the form
$$ \mathcal{P} = \langle \mathtt{Vars}, \mathtt{Values}, \mathtt{Constraints} \rangle $$
where 
- `Vars` is a set of strings which serve as *variables*.

  The idea is that $V_i$ specifies the column of the queen that is placed in row $i$.
    
- `Values` is a set of *values* that can be assigned 
  to the variables in $\mathtt{Vars}$.
  
  In the 8-queens-problem we will have $\texttt{Values} = \{1,\cdots,8\}$.
- `Constraints` is a set of formulas from first order logic.  
  Each of these formulas is  called a *constraint* of $\mathcal{P}$.
  There are two different types of constraints.
  * We have constraints that express that no two queens that are positioned in different rows share the same
    column.  To capture these constraints, we define
    $$\texttt{DifferentCol} := \bigl\{ \texttt{V}_i \not= \texttt{V}_j \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}.$$
    Here the condition $j < i$ ensures that, for example,  while we have the constraint
    $\texttt{V}_2 \not= \texttt{V}_1$ we do not also have the constraint  $\texttt{V}_1 \not= \texttt{V}_2$, as the latter 
    constraint would be redundant if the former constraint had already been established.
  * We have constraints that express that no two queens positioned in different rows share the same 
    diagonal.  The queens in row $i$ and row $j$ share the same diagonal iff the equation
    $$ |i - j| = |V_i - V_j| $$
    holds.  The expression $|i-j|$ is the absolute value of the difference of the rows of the queens in row
    $i$ and row $j$,  while the expression $|V_i - V_j|$ is the absolute value of the difference of the
    columns of these queens.  To capture these constraints, we define
    $$ \texttt{DifferentDiag} := \bigl\{ |i  - j| \not= |\texttt{V}_i - \texttt{V}_j| \bigm| i \in \{1,\cdots,8\} \wedge j \in \{1,\cdots,8\} \wedge j < i \bigr\}. $$

In [ ]:
def create_csp(n):
    S              = range(1, n+1)           
    Variables      = { f'V{i}' for i in S }
    Values         = set(S)
    DifferentCols  = { f'V{i} != V{j}' for i in S
                                       for j in S
                                       if  i < j 
                     }
    DifferentDiags = { f'abs(V{j} - V{i}) != {j - i}' for i in S
                                                      for j in S 
                                                      if  i < j 
                     }
    return Variables, Values, DifferentCols | DifferentDiags

The function `main()` creates a CSP representing the 4-queens puzzle and prints the CSP.
It is included for testing purposes.

In [ ]:
def main():
    Vars, Values, Constraints = create_csp(4)
    print('Variables:  ', Vars)
    print('Values:     ', Values)
    print('Constraints:')
    for c in Constraints:
        print('            ', c)

In [ ]:
main()

## Displaying the Solution

This The following functions have been contributed by Phillip Polland with minor edits from me.

We use the [ElementTree XML API](https://docs.python.org/3.10/library/xml.etree.elementtree.html) below and have to import it.

In [ ]:
import xml.etree.ElementTree as ET

Furthermore, we use the module ipywidgets. They can be installed in the commandline with the following command:

conda install -c conda-forge ipywidge

In [ ]:
from ipywidgets import Layout, HTML

In [ ]:
def _svg(viewbox: int, size: int = None) -> ET.Element:
    svg = ET.Element("svg", {
        "xmlns": "http://www.w3.org/2000/svg",
        "xmlns:xlink": "http://www.w3.org/1999/xlink",
        "version": "1.2",
        "baseProfile": "tiny",
        "viewBox": f"0 0 {viewbox:d} {viewbox:d}",
    })
    if size is not None:
        svg.set("width", str(size))
        svg.set("height", str(size))
    return svg

In [ ]:
class SvgWrapper(str):
    def _repr_svg_(self):
        return self

    def save_svg(self, file):
        with open(file, "w") as f:
            f.write(self)

In [ ]:
def _attrs(attrs):
    return {k: str(v) for k, v in attrs.items() if v is not None}

The function `paint_nxn_board(n, Qs)` paints a chessboard with queens.
* `n` specifies the number of fields of the board.  For a classical chessboard, `n` is eight.
* `Qs` specifies the positions of the queens. `Qs[col]` specifies that in column `col` the queen
  is placed in row `Qs[col]`.

In [ ]:
def paint_nxn_board(n: int = 8, Qs: list = None, SQUARE_SIZE: int = 16, size: int = None, ipy_off=False):
    QUEEN_SVG_STRING = """<g id="white-queen" class="white queen" fill="#fff" fill-rule="evenodd" stroke="#000" stroke-width="1.5" stroke-linecap="round" stroke-linejoin="round"><path d="M8 12a2 2 0 1 1-4 0 2 2 0 1 1 4 0zM24.5 7.5a2 2 0 1 1-4 0 2 2 0 1 1 4 0zM41 12a2 2 0 1 1-4 0 2 2 0 1 1 4 0zM16 8.5a2 2 0 1 1-4 0 2 2 0 1 1 4 0zM33 9a2 2 0 1 1-4 0 2 2 0 1 1 4 0z"/><path d="M9 26c8.5-1.5 21-1.5 27 0l2-12-7 11V11l-5.5 13.5-3-15-3 15-5.5-14V25L7 14l2 12zM9 26c0 2 1.5 2 2.5 4 1 1.5 1 1 .5 3.5-1.5 1-1.5 2.5-1.5 2.5-1.5 1.5.5 2.5.5 2.5 6.5 1 16.5 1 23 0 0 0 1.5-1 0-2.5 0 0 .5-1.5-1-2.5-.5-2.5-.5-2 .5-3.5 1-2 2.5-2 2.5-4-8.5-1.5-18.5-1.5-27 0z" stroke-linecap="butt"/><path d="M11.5 30c3.5-1 18.5-1 22 0M12 33.5c6-1 15-1 21 0" fill="none"/></g>"""  # noqa: E501
    svg = _svg(n * SQUARE_SIZE, size)
    fields = [(file, rank) for file in range(n) for rank in range(n)]
    for file_index, rank_index in fields:
        x = file_index * SQUARE_SIZE
        y = rank_index * SQUARE_SIZE
        light = (file_index + rank_index) % 2 == 0
        cls = ["square", "light" if light else "dark"]
        square_color = "#ffce9e" if light else "#d18b47"
        square_opacity = 1.0
        ET.SubElement(svg, "rect", _attrs({
            "x": x,
            "y": y,
            "width": SQUARE_SIZE,
            "height": SQUARE_SIZE,
            "class": " ".join(cls),
            "stroke": "none",
            "fill": square_color,
            "opacity": square_opacity,
        }))
    if Qs:
        defs = ET.SubElement(svg, "defs")
        defs.append(ET.fromstring(QUEEN_SVG_STRING))
        scale = SQUARE_SIZE / 45
        if any(q >= n or q < 0 for q in Qs if q is not None): raise ValueError("At least one queen is in an impossible field")
        for file_index, rank_index in enumerate(Qs):
            if rank_index is None:
                continue
            x = file_index * SQUARE_SIZE
            y = rank_index * SQUARE_SIZE
            href = f"#white-queen"
            ET.SubElement(svg, "use", _attrs({
                "href": href,
                "xlink:href": href,
                "transform": f"translate({x:d}, {y:d}) scale({scale:f})",
            }))
    svg = SvgWrapper(ET.tostring(svg).decode("utf-8"))
    if not ipy_off:
        html_wrapper = f'<div style="width:25%">{svg}</div>'
        svg_widget = HTML(value=html_wrapper, layout=Layout(grid_area="top"))
        return svg_widget
    return svg

The function `show_solution(Solution)` takes a dictionary that contains a variable assignment that represents a solution to the 8-queens puzzle.  It displays this Solution on a chess board.

In [ ]:
def show_solution(Solution):
    n      = len(Solution)
    Queens = [None for col in range(n)]
    for row in range(n):
        col = Solution['V'+str(row+1)]
        Queens[col-1] = row
    return paint_nxn_board(n, Queens)

In [ ]:
Solution = {'V5': 1, 'V2': 2, 'V4': 6, 'V6': 3, 'V1': 7, 'V7': 5, 'V3': 4}

In [ ]:
show_solution(Solution)